In [1]:
import h5py
import random
import scipy.io
from collections import Counter
from random import shuffle
from numpy import arange
from sklearn.preprocessing import scale
import numpy as np

import importlib
import tsne_functions
importlib.reload(tsne_functions)
from tsne_functions import *

In [2]:
import pylab
import matplotlib as mpl
import IPython
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['figure.dpi'] = 100
IPython.display.set_matplotlib_formats('png','pdf', quality=100)
pylab.rcParams['figure.figsize'] = (5.0, 4.0)

### Load data

In [3]:
data_path='/home/khrovatin/retinal/data/counts/'
data_path_h='/home/khrovatin/retinal/data/human/'

In [4]:
col_data=pd.read_table(data_path+'passedQC_cellData.tsv',index_col=0)
col_by_region=col_data.groupby('region')
fov_cells=col_by_region.get_group('fov').index
per_cells=col_by_region.get_group('per').index

In [7]:
with h5py.File(data_path+'scaledata.h5','r') as file:
    #As h5 file was saved in R it is imported as transposed
    data_int=pd.DataFrame(file.get('integrated/matrix')[:file.get('integrated/matrix').shape[0],:])
    data_int.columns=[name.decode() for name in file.get('integrated/rownames')]
    data_int.index=[name.decode() for name in file.get('integrated/colnames')[:file.get('integrated/colnames').shape[0]]]

In [5]:
with h5py.File(data_path+'normalisedlog.h5','r') as file:    
    data_norm=pd.DataFrame(file.get('ref/matrix')[:file.get('ref/matrix').shape[0],:])
    data_norm.columns=[name.decode() for name in file.get('ref/rownames')]
    data_norm.index=[name.decode() for name in file.get('ref/colnames')[:file.get('ref/colnames').shape[0]]]
    
    data_norm_h=pd.DataFrame(file.get('00012_NeuNM/matrix')[:file.get('00012_NeuNM/matrix').shape[0],:])
    data_norm_h.columns=[name.decode() for name in file.get('00012_NeuNM/rownames')]
    data_norm_h.index=[name.decode() for name in file.get('00012_NeuNM/colnames')[:file.get('00012_NeuNM/colnames').shape[0]]]

In [6]:
# Scale for logistic regression
data_norm=pd.DataFrame(scale(data_norm,with_std=False),index=data_norm.index,columns=data_norm.columns)
data_norm_h=pd.DataFrame(scale(data_norm_h,with_std=False),index=data_norm_h.index,columns=data_norm_h.columns)

In [8]:
counts_h=scipy.io.mmread('/share/LBI_share/public/retina/human snRNAseq data_10x_ChenLab_ZupanLab/humansn_10x_storage/00012_NeuNM_filtered_feature_bc_matrix/matrix.mtx.gz')
counts_h=pd.DataFrame.sparse.from_spmatrix(counts_h).T
counts_h.columns=pd.read_table('/share/LBI_share/public/retina/human snRNAseq data_10x_ChenLab_ZupanLab/humansn_10x_storage/00012_NeuNM_filtered_feature_bc_matrix/features.tsv.gz',header=None)[1]
counts_h.index=pd.read_table('/share/LBI_share/public/retina/human snRNAseq data_10x_ChenLab_ZupanLab/humansn_10x_storage/00012_NeuNM_filtered_feature_bc_matrix/barcodes.tsv.gz',header=None)[0]
counts_h.index=[idx.split('-')[0] for idx in counts_h.index]

### Duplicated gene symbols in human

Some gene symbols are duplicated in the human data (see count below). 

In [9]:
genes_duplicated=[k for k,v in Counter(list(counts_h.columns)).items() if v > 1]
print('Duplicated gene symbols among all human genes', len(genes_duplicated))

Duplicated gene symbols among all human genes 91


In [10]:
#genes_duplicated

In [11]:
genes=pd.read_table('/share/LBI_share/public/retina/human snRNAseq data_10x_ChenLab_ZupanLab/humansn_10x_storage/00012_NeuNM_filtered_feature_bc_matrix/features.tsv.gz',header=None)

In [12]:
#for gene in genes_duplicated:
#    print(genes[genes[1]==gene])
    #print(counts_h.iloc[:,counts_h.columns==gene].sum())

Some of these ENS IDs point to deprecated genes/poorly characterized proteins. As their homology in macaque is uncertain based solely on gene symbols they are removed.

### Retain variable genes with unique symbols and remove low quality cells from human counts

In [13]:
print(data_norm_h.shape,data_norm.shape)

for gene in genes_duplicated:
    if gene in data_norm_h.columns:
          data_norm_h=data_norm_h.drop(gene,axis=1)
    if gene in data_norm.columns:
          data_norm=data_norm.drop(gene,axis=1)

print(data_norm_h.shape,data_norm.shape)

(11194, 2203) (156644, 2203)
(11194, 2196) (156644, 2196)


In [14]:
print(counts_h.shape,data_int.shape)
counts_h=counts_h.loc[data_norm_h.index,data_norm.columns]
data_int=data_int[data_norm.columns]
print(counts_h.shape,data_int.shape)

(11245, 32738) (156644, 3605)
(11194, 2196) (156644, 2196)


In [15]:
markers=pd.read_csv('/home/khrovatin/retinal/data/markers.csv')

In [16]:
genes_retained=set(data_int.columns.values.copy())
genes_marker=set(markers['Marker'])
print('Retained genes:',len(genes_retained),'; marker genes:',len(genes_marker),'; intersection:',
      len(genes_marker & genes_retained))

Retained genes: 2196 ; marker genes: 142 ; intersection: 103


### Split data for classifier evaluation

In [17]:
idx_shuffled=data_int.index.values.copy()
shuffle(idx_shuffled)
split1=int(len(idx_shuffled)*0.8)
split2=int(len(idx_shuffled)*0.9)
train=idx_shuffled[:split1]
validation=idx_shuffled[split1:split2]
test=idx_shuffled[split2:]
print('N train, validation, test:',len(train),len(validation),len(test))

N train, validation, test: 125315 15664 15665


# tSNE embedding 
Uses macaque data obtained with Seurat integration workflow and raw human counts.

## Fit tSNE and KNN
Perform tSNE on macaque data and use KNN with default sklearn parameters for classification.

In [18]:
colour_dicts={'region':{'per':'#478216','fov':'#944368'},
             'cell_type':{'AC':'#bf7195','BC':'#b59a00','EpiImmune':'#75b05b',
                          'HC':'#4d4d4d','PR':'#7a0606','RGC':'#1c5191'}}

In [19]:
data_tsne_train=data_int.loc[train,:]
data_tsne_test=data_int.loc[test,:]

##  tSNE per+fov

### tSNE evaluation on per+fov
Use training set to construct tSNE and classifier. Embed test set on tSNE and use it to evaluate classification. tSNE shows actual classes for both training (lower opacity) and test set.

In [ ]:
tsne_data_int_eval=analyse_tsne(data1=data_tsne_train,data2=data_tsne_test,col_data=col_data,colour_dicts=colour_dicts)

### tSNE on per+fov with all data

In [ ]:
tsne_int=make_tsne(data_int)
#savePickle(data_path+'tSNE_integrated_sharedGenes.pkl',(tsne_int,data_int.index))

In [ ]:
print('By region')
plot_tsne([tsne_int],[dict(zip(col_data.index,col_data['region']))],[ data_int.index], 
         legend=True,plotting_params = {'s': 0.2,'alpha':0.5},colour_dict=colour_dicts['region'])

In [ ]:
print('By cell type')
plot_tsne([tsne_int],[dict(zip(col_data.index,col_data['cell_type']))], [data_int.index],
          legend=True,plotting_params = {'s': 0.2,'alpha':0.5},colour_dict=colour_dicts['cell_type'])

In [ ]:
print('By more specific cell type; there are more types than colours')
plot_tsne([tsne_int],[dict(zip(col_data.index,col_data['cell_types_fine']))], [data_int.index],
          legend=False,plotting_params = {'s': 0.2,'alpha':0.5})

## tSNE fov

Perform tSNE on fov and construct classifier on it. Plots show fov data.

In [ ]:
tsne_int_fov=make_tsne(data_int.loc[fov_cells,:])

In [ ]:
print('By region')
plot_tsne([tsne_int_fov],[dict(zip(col_data.index,col_data['region']))],[ data_int.loc[fov_cells,:].index],
          legend=True,plotting_params = {'s': 0.2,'alpha':0.5},colour_dict=colour_dicts['region'])

In [ ]:
print('By cell type')
plot_tsne([tsne_int_fov],[dict(zip(col_data.index,col_data['cell_type']))], [data_int.loc[fov_cells,:].index],
          legend=True,plotting_params = {'s': 0.2,'alpha':0.5},colour_dict=colour_dicts['cell_type'])

In [ ]:
print('By more specific cell type; there are more types than colours')
plot_tsne([tsne_int_fov],[dict(zip(col_data.index,col_data['cell_types_fine']))], [data_int.loc[fov_cells,:].index],
          legend=False,plotting_params = {'s': 0.2,'alpha':0.5})

### tSNE evaluation of fov with per
Fov tSNE and classifier were used to plot and predict per data. Plots show fov tSNE (less opacity) with added per embedding, both using true (not predicted) classes.

In [ ]:
tsne_data_int_fov=analyse_tsne(data1=data_int.loc[fov_cells,:],data2=data_int.loc[per_cells,:],col_data=col_data,
                               tsne1=tsne_int_fov,colour_dicts=colour_dicts)

## tSNE per

Perform tSNE on per and construct classifier on it. Plots show per data.

In [ ]:
tsne_int_per=make_tsne(data_int.loc[per_cells,:])

In [ ]:
print('By region')
plot_tsne([tsne_int_per],[dict(zip(col_data.index,col_data['region']))],[ data_int.loc[per_cells,:].index],
          legend=True,plotting_params = {'s': 0.2,'alpha':0.5},colour_dict=colour_dicts['region'])

In [ ]:
print('By cell type')
plot_tsne([tsne_int_per],[dict(zip(col_data.index,col_data['cell_type']))], [data_int.loc[per_cells,:].index],
          legend=True,plotting_params = {'s': 0.2,'alpha':0.5},colour_dict=colour_dicts['cell_type'])

In [ ]:
print('By more specific cell type; there are more types than colours')
plot_tsne([tsne_int_per],[dict(zip(col_data.index,col_data['cell_types_fine']))], [data_int.loc[per_cells,:].index],
          legend=False,plotting_params = {'s': 0.2,'alpha':0.5})

### tSNE evaluation of per with fov
Per tSNE and classifier were used to plot and predict fov data. Plots show per tSNE (less opacity) with added fov embedding, both using true (not predicted) classes.

In [ ]:
tsne_data_int_per=analyse_tsne(data1=data_int.loc[per_cells,:],data2=data_int.loc[fov_cells,:],col_data=col_data,
                               tsne1=tsne_int_per,colour_dicts=colour_dicts)

## Human embeding on tSNE
Macaque data (reference), used for tSNE construction, is ploted with less opacity than added human counts data (added). Plot cell_type shows macaque cell types and predicted human cell types based on above described KNN clasifiers. cell_type plot shows human data points in larger size than macaque points. 

### Human counts data on tSNE from fov+per of macaque

In [ ]:
tsne_data_human=embed_tsne_new(data1=data_int,data2=counts_h,col_data1=col_data,tsne1=tsne_int,
                                colour_dicts=colour_dicts)

In [ ]:
savePickle(data_path_h+'tsne_data_human.pkl',(pd.DataFrame(np.array(tsne_data_human[0]),index=data_int.index)
                                              ,pd.DataFrame(np.array(tsne_data_human[1]),index=counts_h.index),
                                          tsne_data_human[2],tsne_data_human[3]))

### Human counts data on tSNE from fov of macaque

In [ ]:
tsne_data_human_fov=embed_tsne_new(data1=data_int.loc[fov_cells,:],data2=counts_h,col_data1=col_data,
                                   tsne1=tsne_int_fov, colour_dicts=colour_dicts)

In [ ]:
savePickle(data_path_h+'tsne_data_human_fov.pkl',(pd.DataFrame(np.array(tsne_data_human_fov[0]),
                                                               index=data_int.loc[fov_cells,:].index)
                                                , pd.DataFrame(np.array(tsne_data_human_fov[1]),
                                                               index=counts_h.index),
                                          tsne_data_human_fov[2],tsne_data_human_fov[3]))

### Human counts data on tSNE from per of macaque

In [ ]:
tsne_data_human_per=embed_tsne_new(data1=data_int.loc[per_cells,:],data2=counts_h,col_data1=col_data,
                                   tsne1=tsne_int_per,colour_dicts=colour_dicts)

In [ ]:
savePickle(data_path_h+'tsne_data_human_per.pkl',(pd.DataFrame(np.array(tsne_data_human_per[0]),
                                                               index=data_int.loc[per_cells,:].index)
                                                , pd.DataFrame(np.array(tsne_data_human_per[1]),
                                                               index=counts_h.index),
                                          tsne_data_human_per[2],tsne_data_human_per[3]))

### Human tSNE
For comparison a tSNE made on human counts data.

In [ ]:
tsne_human=make_tsne(data=counts_h)

In [ ]:
savePickle(data_path_h+'tsne_human_counts.pkl',(pd.DataFrame(np.array(tsne_human),index=counts_h.index)))

In [ ]:
plot_tsne([tsne_human])

# Softmax regression
Uses log normalised CPM data scaled to mean=0 and sd=1 for each gene.

## Fit models

### Choose regularisation parameter
Uses 80% of data for fitting and 10% for validation (in below statistics named as test).

In [ ]:
data_lr_train=data_norm.loc[train,:]
data_lr_validation=data_norm.loc[validation,:]
data_lr_test=data_norm.loc[test,:]

In [ ]:
softmax_train_models=dict()
for c in [0.001,0.01,0.1,1]:
    print('\n********* Regularisation parameter C:',round(c,3))
    m=make_log_regression(data1=data_lr_train,data2=data_lr_validation,col_data=col_data,label='cell_type',
                        logreg={'penalty':'l1','C':c,'random_state':0,'solver':'saga','n_jobs':30})
    softmax_train_models[c]=m

### Make models and evaluate them

In [ ]:
#c=0.01
#softmax_params={'penalty':'l1','C':c,'random_state':0,'solver':'saga','n_jobs':30}

#### Softmax per+fov
Use training and test set to evaluate model, matching above tSNE+KNN classifier.
Make model on whole (unpartitioned) dataset, used for later classification.

In [ ]:
#softmax_all_eval=make_log_regression(data1=data_lr_train,data2=data_lr_test,
#                                     col_data=col_data,label='cell_type',logreg=softmax_params,
#                                   log_reg=softmax_train_models[c])

In [ ]:
#softmax_all=LogisticRegression(**softmax_params).fit(data_norm, col_data.loc[data_norm.index,'cell_type'])

#### Softmax fov
Create softmax model on fov data. Evaluate it with per data.

In [ ]:
#softmax_fov=make_log_regression(data1=data_norm.loc[fov_cells,:],data2=data_norm.loc[per_cells,:],
#                                     col_data=col_data,label='cell_type',logreg=softmax_params)

#### Softmax per
Create softmax model on per data. Evaluate it with fov data.

In [ ]:
#softmax_per=make_log_regression(data1=data_norm.loc[per_cells,:],data2=data_norm.loc[fov_cells,:],
 #                                    col_data=col_data,label='cell_type',logreg=softmax_params)

## Classify human

In [ ]:
#softmax_human=predict(classifier=softmax_all,data=data_norm_h)

In [ ]:
#softmax_human_fov=predict(classifier=softmax_fov,data=data_norm_h)

In [ ]:
#softmax_human_per=predict(classifier=softmax_per,data=data_norm_h)

# Human classification summary

In [ ]:
classified=pd.DataFrame([tsne_data_human[3],tsne_data_human_fov[3],tsne_data_human_per[3],
                         softmax_human,softmax_human_fov,softmax_human_per],
                       index=['knn_all','knn_fov','knn_per','softmax_all','softmax_fov','softmax_per']).T

In [ ]:
main_class=[]
n_main_class=[]
for row in classified.iterrows():
    row=row[1]
    n_main_class.append(row.value_counts().max())
    main_class.append(row.value_counts().idxmax())
classified['main_class']=main_class
classified['N_main_class']=n_main_class